In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.4 MB/s eta 0:00:00


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="7oid83lvHTkuvtigp4im")
project = rf.workspace("knife-yujsw").project("knife-1")
version = project.version(3)
dataset = version.download("yolov11")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 43.9 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to knife-1-3 in yolov11:: 100%|██████████| 11568/11568 [00:02<00:00, 4113.57it/s]


In [ ]:

import os
import shutil
from ultralytics import YOLO
from google.colab import drive

# --- 1. CONFIGURAÇÃO DO GOOGLE DRIVE ---
# drive.mount('/content/drive')  <-- COLOQUE O # AQUI PARA IGNORAR ESSA LINHA
# (Pois você já montou manualmente pelo menu lateral)

# Defina a pasta no seu Drive...
PASTA_DRIVE = '/content/drive/MyDrive/YOLOv11_Small_Augmented'
if not os.path.exists(PASTA_DRIVE):
    os.makedirs(PASTA_DRIVE)
    print(f"✅ Pasta de backup criada no Drive: {PASTA_DRIVE}")
else:
    print(f"✅ Usando pasta existente no Drive: {PASTA_DRIVE}")

# --- 2. FUNÇÃO DE BACKUP AUTOMÁTICO (CALLBACK) ---
def salvar_no_drive(trainer):
    """
    Copia os pesos (best.pt e last.pt) para o Drive ao final de cada época.
    Isso garante que você nunca perca o melhor modelo se o Colab cair.
    """
    pasta_local = os.path.join(trainer.save_dir, 'weights')
    arquivos = ['last.pt', 'best.pt']

    print(f"\n🔄 Sincronizando com o Google Drive (Epoch {trainer.epoch + 1})...")

    for arquivo in arquivos:
        origem = os.path.join(pasta_local, arquivo)
        destino = os.path.join(PASTA_DRIVE, arquivo)

        if os.path.exists(origem):
            try:
                shutil.copy2(origem, destino)
            except Exception as e:
                print(f"⚠️ Erro ao salvar {arquivo}: {e}")
    # print("✅ Backup concluído.")

# --- 3. CONFIGURAÇÃO DO MODELO E TREINO ---

# Carrega o modelo Small (mais inteligente que o Nano)
model = YOLO("yolo11s.pt")

# Adiciona o sistema de backup ao modelo
model.add_callback("on_train_epoch_end", salvar_no_drive)

print("🚀 Iniciando treinamento com Data Augmentation e Backup Automático...")

# Inicia o Treinamento
results = model.train(
    data='/content/knife-1-3/data.yaml',
    epochs=50,
    imgsz=640,

    # --- DATA AUGMENTATION (O SEGREDO PARA ANGULOS) ---
    degrees=90.0,    # Gira as imagens aleatoriamente (até 90 graus)
    flipud=0.5,      # Vira de cabeça para baixo (50% de chance)
    fliplr=0.5,      # Espelha horizontalmente (50% de chance)
    mosaic=1.0,      # Mistura 4 imagens em 1 (ótimo para aprender contexto)
    scale=0.5,       # Zoom in/out aleatório

    # Nome do projeto para organizar os arquivos locais
    name='yolo11s_armas_rotacao'
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Pasta de backup criada no Drive: /content/drive/MyDrive/YOLOv11_Small_Augmented
🚀 Iniciando treinamento com Data Augmentation e Backup Automático...
Ultralytics 8.4.8 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/knife-1-3/data.yaml, degrees=90.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4